In [1]:
# Read all QALD-8 Queries
# For each question
    # Extract all URIs
    # Find sameAs links to DE, FR, ES dbpedia
    # If links exist in same language for all URIs then;
        # generate sparql for all languages
        # write the sparql like: "query_<language-abbr>": { "saprql": <sparql> }
# Store the new QALD File
# 

In [2]:
# Load QALD8 questions

# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://raw.githubusercontent.com/ag-sc/QALD/master/7/data/qald-7-train-multilingual.json"
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())
  
# print the json response
print(data_json['dataset']['id'])

qald-7-train-multilingual


In [ ]:

for question in data_json['questions']:
    sparql = question['query']['sparql']
    temp_res1 = re.findall('res:[^\s\.]+',sparql)
    temp_res1 = [w.replace('res:', 'http://dbpedia.org/resource/') for w in temp_res1]
    temp_res2 = re.findall('http://dbpedia.org/resource/[^>]+', sparql)
    ent_list.update(temp_res1)
    ent_list.update(temp_res2)
    # print(temp_res1, temp_res2)
